<a href="https://colab.research.google.com/github/aliciafmachado/texture_singan/blob/main/SinGAN_Alicia.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Synthesis of textures from generative adversarial networks (SinGAN)

Authors: Alicia FORTES MACHADO et Raphael MENDES

## Introduction

The key innovation on [SinGAN: Learning a Generative Model from a Single Natural Image](https://arxiv.org/pdf/1905.01164.pdf)  is to train from a single image and generalize a GAN to different tasks in computer vision: denoising, inpainting, generation of new images from random noise, and even animation.

[expliquer un peu des autres choses qui sont là aujourdhui]



## Theory

[expliquer comment singan fonctionne et tout]

## Entrainement du réseau de neurones SinGAN

TODO: on peut ajouter des autres images de texture vraiment comme ces proposés au cours

### Setting up

Connecting to drive if you are using colab:

In [1]:
# Connect to drive in to save images and etc directly there
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
%cd 'drive/My Drive/MVA/ImagerieNumerique/projet'
%ls

Mounted at /content/drive
/content/drive/My Drive/MVA/ImagerieNumerique/projet
SinGAN/


Use cells below only if it's first installation.



In [2]:
## Premièrement, on clone le github de singan
! git clone https://github.com/tamarott/SinGAN.git
% cd SinGAN

/content/drive/My Drive/MVA/ImagerieNumerique/projet/SinGAN


In [ ]:
! python -m pip install -r requirements.txt

In [ ]:
# Downgrade torchvision version
! pip install --upgrade torchvision==0.4.0

## Load trained models and test them

In [5]:
%%time
! python random_samples.py --input_name rocks_small_1_8.png --mode random_samples_arbitrary_sizes --gen_start_scale 0 --scale_h 8 --scale_v 8
! python random_samples.py --input_name rocks_small_1_8.png --mode random_samples_arbitrary_sizes --gen_start_scale 0 --scale_h 4 --scale_v 4
! python random_samples.py --input_name rocks_small_1_8.png --mode random_samples_arbitrary_sizes --gen_start_scale 0 --scale_h 2 --scale_v 2
! python random_samples.py --input_name rocks_small_1_8.png --mode random_samples_arbitrary_sizes --gen_start_scale 0 --scale_h 1 --scale_v 1

Random Seed:  3133
Random Seed:  2079
Random Seed:  4853
Random Seed:  5138
CPU times: user 683 ms, sys: 109 ms, total: 793 ms
Wall time: 1min 53s


In [24]:
! python random_samples.py --input_name saumon.png --mode random_samples_arbitrary_sizes --gen_start_scale 0 --scale_h 4 --scale_v 1

Random Seed:  4504


In [25]:
! python random_samples.py --input_name saumon.png --mode random_samples_arbitrary_sizes --gen_start_scale 0 --scale_h 1 --scale_v 4

Random Seed:  9949


In [ ]:
from google.colab import files
# Download Output
! zip -r Output_smaller_rocks_leafs_saumon Output
files.download('Output_smaller_rocks_leafs_saumon.zip')

Here, we can argue that there it has a high complexity with respect to the size of the final image, since it took 3min28s for 8 times the original size of the rocks image. While for 4 it took 52s.

Results are qualitatively comparable to samples generated from larger rocks image.

In terms of metrics, we measure those below.

Training time was similar, which suggests that maybe it trains as much in a smaller image, and therefore improve results. We will try with a larger images and analyze results

In [ ]:
! python paint2image.py --input_name rock_small_cropped.png --ref_name cows.png --paint_start_scale 4

Random Seed:  8663


In [ ]:
! zip -r Output.zip Output

In [27]:
from google.colab import files
files.download('Output.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## Training new models

In [ ]:
! python main_train.py --input_name rocks_small_cropped.png

In [ ]:
%%time
! python main_train.py --input_name rocks_small_1_8.png

In [ ]:
%%time
! python main_train.py --input_name leafs_small_cropped.png

In [ ]:
%%time
! python main_train.py --input_name saumon.png

In [ ]:
%%time
! python main_train.py --input_name leafs_small_1_8.png

In [ ]:
# Can our nn build a wall of bricks? and keep structural things
# TODO

In [ ]:
# Download models
! zip -r TrainedModels_smaller_text.zip TrainedModels
files.download('TrainedModels_smaller_text.zip')

In [ ]:
## On fait des testes
! python random_samples.py --input_name rocks_small.png --mode random_samples --gen_start_scale 0

Random Seed:  6142


We need to convert to JPG so that SIFID works.

In [16]:
! rm Input/Results/rocks_small_1_8.jpg

In [ ]:
filename = 'rocks_small_1_8'
path = "Input/Images"
convert_to_jpg(filename, path, move_to_results=True)

In [ ]:
filename = '0'
path = 'Output/RandomSamples_ArbitrerySizes/rocks_small_1_8/scale_v=4.000000_scale_h=4.000000'
convert_to_jpg(filename, path)

In [13]:
# Remove so that we calculate the SIFID for correct image
! rm Output/RandomSamples_ArbitrerySizes/rocks_small_1_8/scale_v=1.000000_scale_h=1.000000/0.jpg

The path is to the folder and not the file.



In [39]:
%%time
# CAUTION: coment lines 245 and 246 in sifid_score.py to work
# Before running this cell, it's necessary to coment lines 245 and 246 in the code
# Since it requires the last characters of the image to be an integer number
! python SIFID/sifid_score.py --path2real Input/Results --path2fake Output/RandomSamples_ArbitrerySizes/rocks_small_1_8/scale_v\=1.000000_scale_h\=1.000000

100% 1/1 [00:00<00:00, 73.89it/s]
100% 1/1 [00:00<00:00, 154.54it/s]
[0.08457186949316764]
[0.08457187]
SIFID:  0.08457187
CPU times: user 1.09 s, sys: 154 ms, total: 1.24 s
Wall time: 3min 50s


This results is correspondent to the ones reported in the paper. Now let's check for larger outputs.

Then, we compare the 4 times increased to the original image.

In [18]:
%%time
# Evaluate when we want to increase the size of the image 4 times
# horizontally and vertically
! python SIFID/sifid_score.py --path2real Input/Results --path2fake Output/RandomSamples_ArbitrerySizes/rocks_small_1_8/scale_v\=4.000000_scale_h\=4.000000

100% 1/1 [00:00<00:00, 82.29it/s]
100% 1/1 [00:00<00:00,  8.48it/s]
SIFID:  0.74693894
CPU times: user 1 s, sys: 145 ms, total: 1.15 s
Wall time: 3min 41s


In [19]:
! rm Input/Results/rocks_small_1_8.jpg
convert_to_jpg('rocks_small', 'Input/Images', move_to_results=True)

In [20]:
%%time
# Evaluate with respect to original image
! python SIFID/sifid_score.py --path2real Input/Results --path2fake Output/RandomSamples_ArbitrerySizes/rocks_small_1_8/scale_v\=4.000000_scale_h\=4.000000

100% 1/1 [00:00<00:00, 11.92it/s]
100% 1/1 [00:00<00:00,  9.40it/s]
SIFID:  0.2079012
CPU times: user 919 ms, sys: 133 ms, total: 1.05 s
Wall time: 3min 29s


In [22]:
convert_to_jpg('rocks_small_1_8', 'Input/Images', move_to_comp=True)

In [23]:
%%time
# Evaluate patch removed from original image w.r.t. to the one created
! python SIFID/sifid_score.py --path2real Input/Results --path2fake Input/Comparison

100% 1/1 [00:00<00:00, 11.77it/s]
100% 1/1 [00:00<00:00, 103.49it/s]
SIFID:  0.64149064
CPU times: user 926 ms, sys: 131 ms, total: 1.06 s
Wall time: 3min 33s


In [ ]:
# Analyze statistics: distribution...
# TODO: choose statistics and write here
# TODO: implement statistics

In [ ]:
# Transfer learning idea
# Load the model and call training on top of it
# TODO

let'do it!


In [ ]:
! zip -r TrainedModels_w_smaller_rock.zip TrainedModels
# ! zip -r Output_smaller.zip Output
from google.colab import files
files.download('TrainedModels_w_smaller_rock.zip')
# files.download('Output_rock.zip')

In [ ]:
! python main_train.py --input_name leafs_small.png

In [ ]:
! zip -r TrainedModels_leafs.zip TrainedModels
from google.colab import files
files.download('TrainedModels_leafs.zip')

## Résultats qualitatives

In [ ]:
## Faire des plots et tout

## Métriques

On a décidé de faire une analyse quantitative des échantillons générés.

In [ ]:
# On commence par analyser d'une façon naive les image générés
# TODO

In [ ]:
# Maintenant on faire l'analyse des nos résultats avec des differents tailles
def get_metric_naive():
  pass

## Patchmatch et SinGAN

On propose ici une analyse du méthode proprosée à l'article SinGAN en utilisant Patchmatch, de façon à vérifier le méthode.

Le méthode consiste en prendre des patches plus proches des ces qu'on analyse sur l'image produit. De cette façon on peut compare ces patches de l'image généré avec l'image original en utilisant SIFID et en prenant le moyenne à la fin (entre tous les patches de l'image généré).

In [ ]:
# Pour patch-match: comparer les résultats
# Implementation of nearest neighbor
def get_nn_patch():
  pass

In [ ]:
# On calcule les métriques en utilisant get_nn_patch
def get_metric_on_large_image():
  pass

## Auxiliary Functions

In [8]:
from PIL import Image

def resize(img_path, save_path, h=600, v=None):
  im = Image.open(img_path)
  h_or, v_or = im.size
  im_r = im.resize((h, int(v_or * h / h_or)))
  im_r.save(save_path)
  print('Saved image!')

In [21]:
def convert_to_jpg(filename, path, move_to_results=False, move_to_comp=False):
  im = Image.open(path + '/' + filename + '.png')
  rgb_im = im.convert('RGB')
  if move_to_results:
    rgb_im.save('Input/Results/' + filename + '.jpg')
  elif move_to_comp:
    rgb_im.save('Input/Comparison/' + filename + '.jpg')
  else:
    rgb_im.save(path + '/' + filename + '.jpg')

In [9]:
def crop_img(img_path, save_path, size=0.5):
  "Crops from middle."
  im = Image.open(img_path)
  h_or, v_or = im.size
  top = v_or * (1 - size) / 2
  bottom = v_or * (1 + size) / 2
  left = h_or * (1 - size) / 2
  right = h_or * (1 + size) / 2
  im_c = im.crop((left, top, right, bottom))
  im_c.save(save_path)
  print('Cropped image!')

In [13]:
resize(img_path='Input/Images/bricks_original.png', save_path='Input/Images/bricks.png')

Saved image!


In [14]:
crop_img(img_path='Input/Images/bricks.png', save_path='Input/Images/bricks_0_5.png')

Cropped image!
